# Sentiment Analysis Tweet in Bahasa Indonesia

# Import General Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

import re
import string
import requests
import io

C:\Users\Nicholas Sky\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nicholas Sky\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Nicholas Sky\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


# Data Overview

In [2]:
#The dataset that will be used is from GitHub
url = "https://raw.githubusercontent.com/rizalespe/Dataset-Sentimen-Analisis-Bahasa-Indonesia/master/dataset_tweet_sentimen_tayangan_tv.csv"
download = requests.get(url).content

df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df

,Id,Sentiment,Acara TV,Jumlah Retweet,Text Tweet
0,1,positive,HitamPutihTransTV,12,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,2,positive,HitamPutihTransTV,6,Selamat berbuka puasa Semoga amal ibadah hari ...
2,3,positive,HitamPutihTransTV,9,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,4,positive,HitamPutihTransTV,2,selamat ya mas @adietaufan masuk hitamputih
4,5,positive,HitamPutihTransTV,1,Asiknya nonton Hitam Putih Trans7
...,...,...,...,...,...
395,396,negative,MataNajwaMetroTV,0,ini apa banget deh gw paling kesel klo orang2 ...
396,397,negative,MataNajwaMetroTV,0,Orang miskin semakin miskin klo sekolah melaku...
397,398,negative,MataNajwaMetroTV,0,"ga boLeh emosi, cepat tua, nonton #matanajwame..."
398,399,negative,MataNajwaMetroTV,0,dr penampilan saja kyk preman taunya bkin kisr...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              400 non-null    int64 
 1   Sentiment       400 non-null    object
 2   Acara TV        400 non-null    object
 3   Jumlah Retweet  400 non-null    int64 
 4   Text Tweet      400 non-null    object
dtypes: int64(2), object(3)
memory usage: 15.8+ KB


# Data Preprocessing

In [4]:
#Drop unused features
df.drop(['Id', 'Acara TV', 'Jumlah Retweet'], axis = 1, inplace = True)
df

,Sentiment,Text Tweet
0,positive,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,positive,Selamat berbuka puasa Semoga amal ibadah hari ...
2,positive,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,positive,selamat ya mas @adietaufan masuk hitamputih
4,positive,Asiknya nonton Hitam Putih Trans7
...,...,...
395,negative,ini apa banget deh gw paling kesel klo orang2 ...
396,negative,Orang miskin semakin miskin klo sekolah melaku...
397,negative,"ga boLeh emosi, cepat tua, nonton #matanajwame..."
398,negative,dr penampilan saja kyk preman taunya bkin kisr...


In [5]:
#Change the sentiment values to integer
df['Sentiment'] = df['Sentiment'].replace(['positive', 'negative'], [1, 0])
df

,Sentiment,Text Tweet
0,1,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,1,Selamat berbuka puasa Semoga amal ibadah hari ...
2,1,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,1,selamat ya mas @adietaufan masuk hitamputih
4,1,Asiknya nonton Hitam Putih Trans7
...,...,...
395,0,ini apa banget deh gw paling kesel klo orang2 ...
396,0,Orang miskin semakin miskin klo sekolah melaku...
397,0,"ga boLeh emosi, cepat tua, nonton #matanajwame..."
398,0,dr penampilan saja kyk preman taunya bkin kisr...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentiment   400 non-null    int64 
 1   Text Tweet  400 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


In [7]:
# Mengecek Imbalanced Data
df['Sentiment'].value_counts()

1    200
0    200
Name: Sentiment, dtype: int64

# Text Preprocessing

In [8]:
#Import libary for text preprocessing
from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemover import StopWordRemover
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

#We add additional dataset that contains stopwords in Bahasa Indonesia from GitHub
url_stopwords = "https://raw.githubusercontent.com/rizalespe/Dataset-Sentimen-Analisis-Bahasa-Indonesia/master/stopword_tweet_pilkada_DKI_2017.csv"
download_stopwords = requests.get(url_stopwords).content

stopwords = pd.read_csv(io.StringIO(download_stopwords.decode('utf-8')), header = None)

#Create function for text (data) preprocessing
class DataPreprocessing:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        
    def remove_signs(self, feature, new = None):
        def delete_sign(tweet):
            # Remove number in string
            tweet = re.sub(r'[0-9]+', '', tweet)
            # Remove tab, new line, double space and back slice
            tweet = tweet.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"").replace('\s+', " ")
            # Remove non ASCII (emoticon, chinese word, .etc)
            tweet = tweet.encode('ascii', 'replace').decode('ascii')
            # Remove mention, link, hashtag
            tweet = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", tweet).split())
            # Remove incomplete URL
            tweet = tweet.replace("http://", " ").replace("https://", " ")
            # Remove doublespace and doubletick
            return tweet.replace('"', "").replace("'", "").replace("  ", " ")

        sentence = []
        for index, row in self.dataframe.iterrows():
            sentence.append(delete_sign(row[feature]))
        if new:
            self.dataframe[new] = sentence
        else:
            self.dataframe[feature] = sentence
    
    def remove_stopwords(self, feature, new = None):
        factory = StopWordRemoverFactory()

        # Add custom stopwords
        stopword_custom =["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'nyg', 'hehe', 'pen', 'nan', 'loh','&amp', 'yah']
        stopword_custom.extend(stopwords)

        # Add custom stopword to sastrawi and convert to dictionary
        stopword_sastrawi = factory.get_stop_words() + stopword_custom
        dictionary = ArrayDictionary(stopword_sastrawi)

        # Create StopWordRemover Function and add custom stopwords list
        stopword = StopWordRemover(dictionary)

        sentence = []
        for index, row in self.dataframe.iterrows():
            sentence.append(stopword.remove(row[feature]))

        if new:
            self.dataframe[new] = sentence
        else:
            self.dataframe[feature] = sentence

    def text_stemming(self, feature, new = None):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        def stemming(tweet):
            tweet = stemmer.stem(tweet)
            return tweet
        
        sentence = []
        for index, row in self.dataframe.iterrows():
            sentence.append(stemming(row[feature]))

        if new:
            self.dataframe[new] = sentence
        else:
            self.dataframe[feature] = sentence
    
    # Lazy Preprocessing
    def text_preprocessing(self, feature, new = None):
        if new:
            self.remove_signs(feature, new)
            self.remove_stopwords(new, new)
            self.text_stemming(new, new)
        else:
            self.remove_signs(feature)
            self.remove_stopwords(feature)
            self.text_stemming(feature)

In [9]:
#Apply the preprocessing function
df_preprocessing = DataPreprocessing(df)
df_preprocessing.text_preprocessing('Text Tweet', 'Tweet Cleaned')
df

,Sentiment,Text Tweet,Tweet Cleaned
0,1,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S...",undang shanijkt hitamputih menang ssk jkt haru...
1,1,Selamat berbuka puasa Semoga amal ibadah hari ...,selamat buka puasa moga amal ibadah hari ni te...
2,1,"Ada nih di trans7 hitam putih, dia dpt penghar...",ada di trans hitam putih dpt harga di norwegia
3,1,selamat ya mas @adietaufan masuk hitamputih,selamat mas masuk hitamputih
4,1,Asiknya nonton Hitam Putih Trans7,asiknya nonton hitam putih trans
...,...,...,...
395,0,ini apa banget deh gw paling kesel klo orang2 ...,apa banget deh gw paling kesel orang debat pak...
396,0,Orang miskin semakin miskin klo sekolah melaku...,orang miskin makin miskin sekolah laku pungut ...
397,0,"ga boLeh emosi, cepat tua, nonton #matanajwame...",boleh emosi cepat tua nonton lihat bapak tiba ...
398,0,dr penampilan saja kyk preman taunya bkin kisr...,dr tampil kyk preman tau bkin kisruh usak


# Split the data

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Tweet Cleaned'], df['Sentiment'], shuffle=True, test_size=0.2, stratify=df['Sentiment'], random_state=20)

# Text Vectorization

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Count Vectorize
count_vector = CountVectorizer(max_features=10000)
count_vector.fit_transform(X_train)

# TF-IDF
tfidf_vector = TfidfVectorizer(max_features=10000)
tfidf_vector.fit_transform(X_train)

# For this case, we will use the TF-IDF
X_train = tfidf_vector.transform(X_train)
X_test = tfidf_vector.transform(X_test)

# Modelling

In [12]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# Init for SVM
svm = svm.SVC(kernel="linear")

def eval_classification(model, pred, xtrain, ytrain, xtest, ytest):
    # Init List
    fpr, tpr, thresholds = roc_curve (ytest, pred, pos_label=1) # pos_label is positive label for metric, default is 1
    acc = "%.4f" % accuracy_score(ytest, pred)
    prc = "%.4f" % precision_score(ytest, pred)
    rec = "%.4f" % recall_score(ytest, pred)
    f1s = "%.4f" % f1_score(ytest, pred)
    aucs = "%.4f" % auc(fpr, tpr)
    data = [['Accuracy', acc], ['Precision', prc], ['Recall', rec], ['F1-Score', f1s], ['AUC', aucs]]

    # Create DataFrame of the metrics
    summary = pd.DataFrame(data, columns=['Metric', 'Value'])

    return summary

## Logistic Regression

In [13]:
logreg = LogisticRegression()

imp_logreg = logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)

summary = eval_classification(imp_logreg, logreg_pred, X_train, y_train, X_test, y_test)
summary

,Metric,Value
0,Accuracy,0.8000
1,Precision,0.8158
2,Recall,0.7750
3,F1-Score,0.7949
4,AUC,0.8000


## K-Nearest Neighbor (KNN)

In [14]:
k=15
knn = KNeighborsClassifier(n_neighbors=k)

imp_knn = knn.fit(X_train, y_train)
knn_pred = imp_knn.predict(X_test)

summary = eval_classification(imp_knn, knn_pred, X_train, y_train, X_test, y_test)
summary

,Metric,Value
0,Accuracy,0.7000
1,Precision,0.6538
2,Recall,0.8500
3,F1-Score,0.7391
4,AUC,0.7000


## Support Vector Machine (SVM)

In [15]:
imp_svm = svm.fit(X_train, y_train)
svm_pred = imp_svm.predict(X_test)

summary = eval_classification(imp_svm, svm_pred, X_train, y_train, X_test, y_test)
summary

,Metric,Value
0,Accuracy,0.8125
1,Precision,0.8205
2,Recall,0.8000
3,F1-Score,0.8101
4,AUC,0.8125


# Data Testing

In [16]:
class TweetPreprocessing:
    def __init__(self, tweet, vector):
        self.tweet = tweet
        self.vector = vector

    # THIS FUNCTION FOR PREPROCESSING TEXT IN DATAFRAME
    def remove_signs(self, tweet):
        # Remove number in string
        tweet = re.sub(r'[0-9]+', '', tweet)
        # Remove tab, new line, double space and back slice
        tweet = tweet.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"").replace('\s+', " ")
        # Remove non ASCII (emoticon, chinese word, .etc)
        tweet = tweet.encode('ascii', 'replace').decode('ascii')
        # Remove mention, link, hashtag
        tweet = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", tweet).split())
        # Remove incomplete URL
        tweet = tweet.replace("http://", " ").replace("https://", " ")
        # Remove doublespace and doubletick
        return tweet.replace('"', "").replace("'", "").replace("  ", " ")
    
    def remove_stopwords(self, tweet):
        factory = StopWordRemoverFactory()

        # You can custom stopwords list below, we will use stopword custom for remove stopword
        stopword_custom =["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'nyg', 'hehe', 'pen', 'nan', 'loh','&amp', 'yah']
        stopword_custom.extend(stopwords)

        # Add custom stopword to sastrawi and convert to dictionary
        stopword_sastrawi = factory.get_stop_words() + stopword_custom
        dictionary = ArrayDictionary(stopword_sastrawi)

        # Create StopWordRemover Function and add custom stopwords list
        stopword = StopWordRemover(dictionary)

        tweet = stopword.remove(tweet)
        return tweet

    def text_stemming(self, tweet):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        tweet = stemmer.stem(tweet)
        return tweet

    def vectorizer(self, tweet, vector = tfidf_vector) :
        output = vector.transform([tweet])
        return output

    # Lazy Preprocessing
    def text_preprocessing(self):
        proc_sign = self.remove_signs(self.tweet)
        proc_stop = self.remove_stopwords(proc_sign)
        proc_stem = self.text_stemming(proc_stop)
        proc_vect = self.vectorizer(proc_stem, self.vector)
        return proc_vect

## Logistic Regression

In [17]:
comment = input("Comment: ")

# Init Preprocessing
process = TweetPreprocessing(comment, tfidf_vector)
text = process.text_preprocessing()

# Predicting Sentiment
result_logreg = imp_logreg.predict(text)
if result_logreg[0] == 0:
    print('Negative')
else:
    print('Positive')

Comment: anjay keren om
Positive


## K-Nearest Neighbor (KNN)

In [18]:
comment = input("Comment: ")

# Init Preprocessing
process = TweetPreprocessing(comment, tfidf_vector)
text = process.text_preprocessing()

# Predicting Sentiment
result_knn = imp_knn.predict(text)
if result_knn[0] == 0:
    print('Negative')
else:
    print('Positive')

Comment: lah kamu kok telat
Negative


## Support Vector Machine (SVM)

In [19]:
comment = input("Comment: ")

# Init Preprocessing
process = TweetPreprocessing(comment, tfidf_vector)
text = process.text_preprocessing()

# Predicting Sentiment
result_svm = imp_svm.predict(text)
if result_svm[0] == 0:
    print('Negative')
else:
    print('Positive')

Comment: dasar anak kurang ajar
Negative


# Saving The Models

In [22]:
import joblib

joblib.dump(imp_logreg, 'Logistic_Regression_Classifier.pkl')
joblib.dump(imp_knn, 'KNN_Classifier.pkl')
joblib.dump(imp_svm, 'SVM_Classifier.pkl')

['SVM_Classifier.pkl']